In [1]:
# 用于计算每个用户对于所有项目的评分，并做排序
# 由于权重不断会被更新，在线完成比较好，离线计算也可
import pandas as pd
import numpy as np

In [5]:
tag_df = pd.DataFrame(pd.read_csv('./items_tags.csv'))
weight_df = pd.DataFrame(pd.read_csv('./user_weight.csv'))

In [6]:
pd.set_option('display.max_columns', None)
tag_df.head()

,id,code,近一周,一周到一个月,一个月以上,其他岗位,管培生岗位,数据算法,开发测试运维,游戏类岗位,售前售后技术支持,金融财务,互联网运营,互联网产品,营销销售岗,制造业岗位,其他,北京,上海,广州,深圳,杭州,武汉,成都,贵阳,大连,沈阳,其他.1,本科,硕士,大专,实习,全职
0,1,1060112,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,1070012,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,17,1080412,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,3,1070412,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,4,1020412,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [7]:
items = tag_df.iloc[:,2:33]
items_id = tag_df.iloc[:,0]
# items_id
items.head()

,近一周,一周到一个月,一个月以上,其他岗位,管培生岗位,数据算法,开发测试运维,游戏类岗位,售前售后技术支持,金融财务,互联网运营,互联网产品,营销销售岗,制造业岗位,其他,北京,上海,广州,深圳,杭州,武汉,成都,贵阳,大连,沈阳,其他.1,本科,硕士,大专,实习,全职
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [8]:
# 这里只选 5 个模拟用户，来降低运算时间
user = weight_df.iloc[0:5,1:32]
user

,近一周,一周到一个月,一个月以上,其他岗位,管培生岗位,数据算法,开发测试运维,游戏类岗位,售前售后技术支持,金融财务,互联网运营,互联网产品,营销销售岗,制造业岗位,其他,北京,上海,广州,深圳,杭州,武汉,成都,贵阳,大连,沈阳,其他.1,本科,硕士,大专,实习,全职
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,10.0,1.0,10.0,1.0
1,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,10.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,10.0,1.0,1.0,10.0
3,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,10.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,10.0,1.0,1.0,10.0


In [9]:
user_np = user.values
item_np = items.values
print(item_np)

[[1. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 1.]
 ...
 [1. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 1. 0.]]


In [10]:
item_np_T = item_np.T

In [11]:
# 权重和标签 X乘
score = np.matmul(user_np,item_np_T).T
score_df=pd.DataFrame.from_records(score)
score_df.head()

,0,1,2,3,4
0,5.0,18.0,19.0,9.0,14.0
1,4.0,17.0,13.0,8.0,13.0
2,8.0,18.0,14.0,9.0,17.0
3,5.0,18.0,14.0,9.0,14.0
4,5.0,22.0,14.0,13.0,14.0


In [12]:
# 将ID 合并过来
frames=[items_id,score_df]
score_list=pd.concat(frames, axis=1)
score_list.head()

,id,0,1,2,3,4
0,1,5.0,18.0,19.0,9.0,14.0
1,2,4.0,17.0,13.0,8.0,13.0
2,17,8.0,18.0,14.0,9.0,17.0
3,3,5.0,18.0,14.0,9.0,14.0
4,4,5.0,22.0,14.0,13.0,14.0


In [13]:
# 按某个人的做一个排序
sorted_df = score_list.sort_values(by=0,ascending = False)
print (sorted_df)

        id     0     1     2     3     4
650    652  23.0   9.0  14.0  18.0  14.0
2216  2218  23.0   5.0  14.0  14.0  14.0
643    645  23.0   9.0  14.0  18.0  14.0
409    411  23.0   9.0  14.0  18.0  14.0
2221  2223  23.0   5.0  14.0  14.0  14.0
1844  1846  23.0   5.0  14.0  14.0  14.0
607    609  23.0   9.0  14.0  18.0  14.0
2218  2220  23.0   5.0  14.0  14.0  14.0
2183  2185  23.0   5.0  14.0  14.0  14.0
609    611  23.0   9.0  14.0  18.0  14.0
866    868  23.0   5.0  14.0  14.0  14.0
599    601  23.0   9.0  14.0  18.0  14.0
2211  2213  23.0   5.0  14.0  14.0  14.0
2210  2212  23.0   5.0  14.0  14.0  14.0
307    309  23.0   9.0  14.0  18.0  14.0
524    526  23.0   9.0  14.0  18.0  14.0
1658  1660  23.0   5.0  14.0  14.0  14.0
506    508  23.0   9.0  14.0  18.0  14.0
621    623  23.0   9.0  14.0  18.0  14.0
728    730  23.0   9.0  14.0  18.0  14.0
406    408  23.0   9.0  14.0  18.0  14.0
596    598  23.0   9.0  14.0  18.0  14.0
919    921  23.0   5.0  14.0  14.0  14.0
577    579  23.0